In [104]:
import pandas as pd
import numpy as np
from ast import literal_eval

In [105]:
df = pd.read_csv("../cleaned_data/movies_main.csv")

In [106]:
df

,Id,Original_Title,Original_Language,Spoken_Languages,Budget,Revenue,Runtime,Release_Date,Production_Companies,Production_Countries,...,Overview,Genres,Genres_Parse,Belongs_To_Collection_Parse,Spoken_Languages_Parse,Production_Companies_Parse,Production_Countries_Parse,Production_Countries_Code_Parse,Keywords,Keywords_parse
0,862,Toy Story,en,"[{'iso_639_1': 'en', 'name': 'English'}]",30000000.0,373554033.0,81.0,1995-10-30,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",...,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","['Animation', 'Comedy', 'Family']",['Toy Story Collection'],['English'],['Pixar Animation Studios'],['United States of America'],['US'],"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","['jealousy', 'toy', 'boy', 'friendship', 'frie..."
1,8844,Jumanji,en,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",65000000.0,262797249.0,104.0,1995-12-15,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","['Adventure', 'Fantasy', 'Family']",NaN,"['English', 'Français']","['TriStar Pictures', 'Teitler Film', 'Intersco...",['United States of America'],['US'],"[{'id': 10090, 'name': 'board game'}, {'id': 1...","['board game', 'disappearance', ""based on chil..."
2,15602,Grumpier Old Men,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,0.0,101.0,1995-12-22,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","['Romance', 'Comedy']",['Grumpy Old Men Collection'],['English'],"['Warner Bros.', 'Lancaster Gate']",['United States of America'],['US'],"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","['fishing', 'best friend', 'duringcreditssting..."
3,31357,Waiting to Exhale,en,"[{'iso_639_1': 'en', 'name': 'English'}]",16000000.0,81452156.0,127.0,1995-12-22,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",...,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","['Comedy', 'Drama', 'Romance']",NaN,['English'],['Twentieth Century Fox Film Corporation'],['United States of America'],['US'],"[{'id': 818, 'name': 'based on novel'}, {'id':...","['based on novel', 'interracial relationship',..."
4,11862,Father of the Bride Part II,en,"[{'iso_639_1': 'en', 'name': 'English'}]",0.0,76578911.0,106.0,1995-02-10,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",...,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",['Comedy'],['Father of the Bride Collection'],['English'],"['Sandollar Productions', 'Touchstone Pictures']",['United States of America'],['US'],"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","['baby', 'midlife crisis', 'confidence', 'agin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45428,41811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
45429,279687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
45430,414453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],[]
45431,87229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 5746, 'name': 'iceland'}, {'id': 23175...","['iceland', 'coming home']"


In [107]:
df_genres_matrix = df.copy()

In [108]:
df_genres_matrix = df_genres_matrix[["Id", "Genres_Parse"]]

In [109]:
df_genres_matrix

,Id,Genres_Parse
0,862,"['Animation', 'Comedy', 'Family']"
1,8844,"['Adventure', 'Fantasy', 'Family']"
2,15602,"['Romance', 'Comedy']"
3,31357,"['Comedy', 'Drama', 'Romance']"
4,11862,['Comedy']
...,...,...
45428,41811,NaN
45429,279687,NaN
45430,414453,NaN
45431,87229,NaN


In [110]:
def parse_genre_to_list(genre_list):
    if genre_list != "nan" and pd.notnull(genre_list):
        return literal_eval(genre_list)
    else:
        return []

In [111]:
df_genres_matrix["Genres_Parse"] = df_genres_matrix["Genres_Parse"].map(lambda x : parse_genre_to_list(x))

In [112]:
df_genres_matrix

,Id,Genres_Parse
0,862,"[Animation, Comedy, Family]"
1,8844,"[Adventure, Fantasy, Family]"
2,15602,"[Romance, Comedy]"
3,31357,"[Comedy, Drama, Romance]"
4,11862,[Comedy]
...,...,...
45428,41811,[]
45429,279687,[]
45430,414453,[]
45431,87229,[]


In [113]:
type(df_genres_matrix.loc[1, 'Genres_Parse'])

list

In [114]:
def matrix_conversion_per_row(row):
    print(row["Genres_Parse"])
    #for i in row["Genres_Parse"]:
        #df_genres_matrix.loc[row.index, str(i)] = 1
        #print(i)

In [115]:
df_genres_matrix.head(10).map(lambda x : matrix_conversion_per_row(x["Genres_Parse"]))

TypeError: 'int' object is not subscriptable

In [ ]:
df_genres_matrix["Genres_Parse"].explode()

In [117]:
# Step 1: Explode the list of genres
exploded_df = df_genres_matrix.explode('Genres_Parse')

# Step 2: One-hot encode the genres
one_hot = pd.get_dummies(exploded_df['Genres_Parse'])

# Step 3: Group by movie_id (or the original index) and aggregate with max
genre_df = one_hot.groupby(exploded_df.index).max()

# Step 4: Concatenate the original DataFrame with the new genre columns
df_genres = pd.concat([df_genres_matrix, genre_df], axis=1)

# Display the final DataFrame
df_genres.head()

,Id,Genres_Parse,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,862,"[Animation, Comedy, Family]",False,False,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
1,8844,"[Adventure, Fantasy, Family]",False,True,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,15602,"[Romance, Comedy]",False,False,False,True,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,31357,"[Comedy, Drama, Romance]",False,False,False,True,False,False,True,False,...,False,False,False,False,True,False,False,False,False,False
4,11862,[Comedy],False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
